In [7]:
import nltk
from nltk.collocations import *
from nltk.stem import WordNetLemmatizer
import re
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [33]:
punct = '.,!?():;«»'
#корпус про судебные разбирательства, 174723 токенов
with open('news.txt', 'r', encoding='utf-8') as f:
    text = f.read()
words = text.split()
words = [word.strip(punct) for word in words]
words[:100]

['Губернаторы',
 'отвыкли',
 'от',
 'публичной',
 'политики',
 'и',
 'их',
 'придется',
 'учить',
 'общаться',
 'с',
 'избирателями',
 '—',
 'об',
 'этом',
 'говорили',
 'на',
 'встрече',
 'в',
 'Кремле',
 'с',
 'политологами',
 'Вячеслав',
 'Володин',
 'и',
 'другие',
 'кураторы',
 'внутренней',
 'политики',
 'Но',
 'недовольство',
 'руководства',
 'страны',
 'вызывают',
 'только',
 'главы',
 'тех',
 'регионов',
 'где',
 'нет',
 'реальной',
 'угрозы',
 'власти',
 'объясняет',
 'участник',
 'встречи',
 'С',
 'реальными',
 'оппонентами',
 'местных',
 'властей',
 'Кремль',
 'сам',
 'помогает',
 'бороться',
 'На',
 'встречу',
 'с',
 'первым',
 'замглавы',
 'администрации',
 'президента',
 'Вячеславом',
 'Володиным',
 'пригласили',
 'около',
 '30',
 'политологов',
 'в',
 'первую',
 'очередь',
 'тех',
 'кто',
 'назначен',
 'Кремлем',
 'наблюдать',
 'за',
 'ходом',
 'избирательных',
 'кампаний',
 'на',
 'местах',
 'рассказали',
 'РБК',
 'несколько',
 'участников',
 'встречи',
 'По',
 'словам

In [15]:
lmtzr = WordNetLemmatizer()
words_tagged = []
for word in words:
    tagged = lmtzr.lemmatize(word)
    words_tagged.append(tagged)

In [34]:
finder = BigramCollocationFinder.from_words(words_tagged)

In [82]:
#First metric Likelihood Ratio, bigrams with frequency >=3
#ни одна из коллокаций в топ-10 не вошла в мой ЗС, из топ-10 хорошими коллокациями я считаю ('млрд', 'руб.'), 
#('Кроме', 'того,'), ('главный', 'экономист'), ('точки', 'зрения'), ('сказал', 'РБК'), ('рассказал', 'РБК'), ('млн', 'руб.')
#из моего ЗС в топ-10000 попали 6 коллокаций
stopwords = nltk.corpus.stopwords.words('russian')
finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in stopwords)
print(finder.nbest(bigram_measures.likelihood_ratio, 10))
lhr = finder.nbest(bigram_measures.likelihood_ratio, 10000)

[('млрд', 'руб.'), ('млрд', 'руб.,'), ('Кроме', 'того,'), ('главный', 'экономист'), ('точки', 'зрения'), ('сказал', 'РБК'), ('источник,', 'близкий'), ('млн', 'руб.'), ('АФК', '«Система»'), ('рассказал', 'РБК')]


In [84]:
#Second metric  PMI, bigrams with frequency >=3 and only Russian words
#ни одна из коллокаций в топ-10 не вошла в мой ЗС, из топ-10 хорошими коллокациями я считаю  ('железным', 'дорогам»'), 
#('сберегательные', 'сертификаты'), ('свободное', 'плавание'), ('«Петербургская', 'политика»'), ('«Фольксваген', 'Груп'),
#('«негативный', 'консенсус»')
#из моего ЗС в топ-10000 попали 6 коллокаций
stopwords = nltk.corpus.stopwords.words('russian')
finder.apply_word_filter(lambda w: len(w) < 3 or re.search('[а-яё]+', w.lower()) is None or w.lower() in stopwords)
finder.apply_freq_filter(3)
print(finder.nbest(bigram_measures.pmi, 10))
pmi = finder.nbest(bigram_measures.pmi, 10000)

[('«Российским', 'железным'), ('«негативный', 'консенсус»'), ('Белорусской', 'калийной'), ('железным', 'дорогам»'), ('сберегательные', 'сертификаты'), ('свободное', 'плавание'), ('«Петербургская', 'политика»'), ('«Фольксваген', 'Груп'), ('Груп', 'Рус»'), ('Общероссийского', 'народного')]


In [72]:
#Мой ЗС
from nltk.metrics.spearman import *
results_list = [('официальный', 'представитель'), ('заявил', 'премьер-министр'), ('со', 'стороны'), ('Центральный', 'банк'), 
                ('сообщил', 'РБК'), ('решение', 'суда'), ('развитие', 'системы'), ('условиях', 'санкций'), 
                ('противном', 'случае'), ('размещать', 'средства')]

In [79]:
#Посмотрите, какую позицию занимают коллокации из золотого стандарта среди списка, отранжированного 
#по мере коллокационной связи.
for coll in results_list:
    if coll in lhr:
        print ('lhr', coll, lhr.index(coll))
    if coll in pmi:
        print ('pmi', coll, pmi.index(coll))

lhr ('официальный', 'представитель') 360
pmi ('официальный', 'представитель') 1306
lhr ('Центральный', 'банк') 912
pmi ('Центральный', 'банк') 1080
lhr ('сообщил', 'РБК') 32
pmi ('сообщил', 'РБК') 1720
lhr ('решение', 'суда') 1780
pmi ('решение', 'суда') 1619
lhr ('условиях', 'санкций') 849
pmi ('условиях', 'санкций') 1603
lhr ('противном', 'случае') 1413
pmi ('противном', 'случае') 1106


In [80]:
#Spearman with first metrik's 10000 best

print('%0.1f' % spearman_correlation(ranks_from_sequence(results_list), ranks_from_sequence(lhr)))

-194007.0


In [81]:
#Spearman with second metrik's 10000 best
print('%0.1f' % spearman_correlation(ranks_from_sequence(results_list), ranks_from_sequence(pmi)))

-347657.2
